In [ ]:
import pandas as pd
import numpy as np

import	sys
import os

import pypdf
import pdfplumber
import re
import os

In [18]:
# Load the PDF
pdf_path = "250429-1q-2025-earnings-release-investors-and-analysts-call-transcript.pdf"

# Extract Year and Quarter from heading
with pdfplumber.open(pdf_path) as pdf:
    first_page_text = pdf.pages[0].extract_text()

quarter_match = re.search(r"\b(Q[1-4])\s+(\d{4})\b", first_page_text)
if quarter_match:
    quarter_extracted = quarter_match.group(1)
    year_extracted = int(quarter_match.group(2))
else:
    quarter_extracted = "Unknown"
    year_extracted = None

# Initialize variables
qa_data_combined = []
current_speaker = ""
current_institution = ""
current_text = []

# Parse the Q&A portion of the document
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue
        lines = text.split("\n")

        for line in lines:
            # Match either: FULL format (with institution) or SHORT format (no institution)
            match_full = re.match(r"^([A-Z\s\.]+),\s*([A-Za-z\s&]+):\s*(.*)", line)
            match_short = re.match(r"^([A-Z\s\.]+):\s*(.*)", line)

            if match_full:
                # Save previous speaker
                if current_text:
                    qa_data_combined.append({
                        "Bank Name": "HSBC",
                        "Year": year_extracted,
                        "Quarter": quarter_extracted,
                        "Speaker name": current_speaker,
                        "Institution": current_institution,
                        "Speaker text": " ".join(current_text),
                        "flag_question": any("?" in t for t in current_text)
                    })
                    current_text = []

                # New speaker
                current_speaker = match_full.group(1).title()
                current_institution = match_full.group(2).title()
                remainder = match_full.group(3).strip()
                if remainder:
                    current_text.append(remainder)

            elif match_short:
                # Save previous speaker
                if current_text:
                    qa_data_combined.append({
                        "Bank Name": "HSBC",
                        "Year": year_extracted,
                        "Quarter": quarter_extracted,
                        "Speaker name": current_speaker,
                        "Institution": current_institution,
                        "Speaker text": " ".join(current_text),
                        "flag_question": any("?" in t for t in current_text)
                    })
                    current_text = []

                # New internal speaker
                current_speaker = match_short.group(1).title()
                current_institution = ""
                remainder = match_short.group(2).strip()
                if remainder:
                    current_text.append(remainder)

            else:
                if line.strip():
                    current_text.append(line.strip())

# Capture the last block
if current_text:
    qa_data_combined.append({
        "Bank Name": "HSBC",
        "Year": year_extracted,
        "Quarter": quarter_extracted,
        "Speaker name": current_speaker,
        "Institution": current_institution,
        "Speaker text": " ".join(current_text),
        "flag_question": any("?" in t for t in current_text)
    })

# Create and save DataFrame
df_combined_qa = pd.DataFrame(qa_data_combined)

#remove row with empty Speaker name
df_combined_qa = df_combined_qa[df_combined_qa['Speaker name'].str.strip() != ""]


df_combined_qa.to_csv("hsbc_q1_2025_combined_qa.csv", index=False)

# Preview
print(df_combined_qa.head())


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

  Bank Name  Year Quarter      Speaker name Institution  \
1      HSBC  2025      Q1  Georges Elhedery   Group Ceo   
2      HSBC  2025      Q1          Pam Kaur   Group Cfo   
3      HSBC  2025      Q1     Benjamin Toms         Rbc   
4      HSBC  2025      Q1  Georges Elhedery               
5      HSBC  2025      Q1          Pam Kaur               

                                        Speaker text  flag_question  
1  Welcome, all, to today’s call. I’m joined here...          False  
2  Thank you, Georges. Thank you, everyone, for j...           True  
3  Good morning, both, and thank you for taking m...           True  
4  Thank you very much, Ben. So we’ve announced, ...          False  
5  Ben, just to add, for the quarter, there’s bee...          False  


In [1]:
# ---------- FUNCTION TO PARSE ONE PDF FILE ----------
def extract_qa_from_pdf(pdf_path, file_label=None):
    qa_data = []
    current_speaker = ""
    current_institution = ""
    current_text = []

    try:
        with pdfplumber.open(pdf_path) as pdf:
            first_page_text = pdf.pages[0].extract_text()

            # ------------------ YEAR DETECTION ------------------
            year = None
            year_match = re.search(r"\b(20[1-3][0-9])\b", first_page_text)
            if year_match:
                year = int(year_match.group(1))

            # ------------------ QUARTER DETECTION ------------------
            quarter = "Unknown"
            quarter_text = first_page_text.lower()

            # Priority: Post-Results + trailing descriptor
            post_results_match = re.search(r"post-results\s+([A-Za-z\s\-]+)", first_page_text, re.IGNORECASE)
            if post_results_match:
                quarter = "Post-Results " + post_results_match.group(1).strip()
            elif re.search(r"\bq\s*1\b", quarter_text):
                quarter = "Q1"
            elif re.search(r"\bq\s*2\b", quarter_text):
                quarter = "Q2"
            elif re.search(r"\bq\s*3\b", quarter_text):
                quarter = "Q3"
            elif re.search(r"\bq\s*4\b", quarter_text):
                quarter = "Q4"
            elif re.search(r"interim", quarter_text) and re.search(r"\bh[\s\-]*1\b", quarter_text):
                quarter = "Interim/H1"
            elif re.search(r"interim", quarter_text):
                quarter = "Interim"
            elif re.search(r"\bh[\s\-]*1\b", quarter_text):
                quarter = "H1"
            elif re.search(r"\bannual\b|\bfull\s*year\b", quarter_text):
                quarter = "Annual"
            # No fallback to "Annual" based on "results"

            # ------------------ EXTRACT SPEAKER TEXT ------------------
            for page in pdf.pages:
                text = page.extract_text()
                if not text:
                    continue
                lines = text.split("\n")

                for line in lines:
                    match_full = re.match(r"^([A-Z\s\.]+),\s*([A-Za-z\s&]+):\s*(.*)", line)
                    match_short = re.match(r"^([A-Z\s\.]+):\s*(.*)", line)

                    if match_full:
                        if current_text:
                            qa_data.append({
                                "File": file_label or os.path.basename(pdf_path),
                                "Bank Name": "HSBC",
                                "Year": year,
                                "Quarter": quarter,
                                "Speaker name": current_speaker,
                                "Institution": current_institution,
                                "Speaker text": " ".join(current_text),
                                "flag_question": any("?" in t for t in current_text)
                            })
                            current_text = []

                        current_speaker = match_full.group(1).title()
                        current_institution = match_full.group(2).title()
                        remainder = match_full.group(3).strip()
                        if remainder:
                            current_text.append(remainder)

                    elif match_short:
                        if current_text:
                            qa_data.append({
                                "File": file_label or os.path.basename(pdf_path),
                                "Bank Name": "HSBC",
                                "Year": year,
                                "Quarter": quarter,
                                "Speaker name": current_speaker,
                                "Institution": current_institution,
                                "Speaker text": " ".join(current_text),
                                "flag_question": any("?" in t for t in current_text)
                            })
                            current_text = []

                        current_speaker = match_short.group(1).title()
                        current_institution = ""
                        remainder = match_short.group(2).strip()
                        if remainder:
                            current_text.append(remainder)
                    else:
                        if line.strip():
                            current_text.append(line.strip())

            # Append the last speaker block
            if current_text:
                qa_data.append({
                    "File": file_label or os.path.basename(pdf_path),
                    "Bank Name": "HSBC",
                    "Year": year,
                    "Quarter": quarter,
                    "Speaker name": current_speaker,
                    "Institution": current_institution,
                    "Speaker text": " ".join(current_text),
                    "flag_question": any("?" in t for t in current_text)
                })

    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

    return qa_data


# ---------- PROCESS ALL PDFs IN FOLDER (INCLUDING SUBFOLDERS) ----------
def process_all_pdfs(root_dir):
    all_results = []

    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(".pdf"):
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, root_dir)
                print(f"Processing: {rel_path}")
                qa_rows = extract_qa_from_pdf(full_path, file_label=rel_path)
                all_results.extend(qa_rows)

    df_all = pd.DataFrame(all_results)
    df_all.to_csv("combined_all_qas.csv", index=False)
    return df_all


# ---------- USAGE ----------
# Update the path to your folder
df = process_all_pdfs(r"C:\Users\User\Desktop\CAM_BoE_GitHub\team-42\HSBC Presentation texts")


NameError: name 'os' is not defined